<p><h1>Segmenting and Clustering Neighborhoods in Toronto</h1></p>
<p><b>Name:</b> Felipe Cassaro Frazatto</p>
<p>This notebook will cover <b>all three parts</b> of the proposed assignmet.</p>
<p>Each part will have a label and a brief description of what is proposed on that part. I'll comment each code part as well.</p>


<h3>Install Libraries</h3>

In [212]:
###Uncomment if some libraries are not installed

##For math and more
#!pip install pandas
#!pip install numpy

##For Plotting
#!pip install matplotlib
#!pip install seaborn
#!pip install folium

##For Algorithms and Machine Learning
#!pip install sklearn

#!pip install lxml html5lib beautifulsoup4
!pip3 install geopy



     |████████████████████████████████| 111 kB 1.3 MB/s eta 0:00:01


<h3>Import Libraries</h3>

In [254]:
import pandas as pd
import folium 
import requests
import json
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from sklearn.cluster import KMeans

<p><h2>First Part</h2></p>
<p><h3>Data Scraping</h3></p>
<p>Read Wikipedia's table of Canada Postal Codes, clean and format it.</p>
<p>1. Download wikipedia data and store postal code table.</p>

In [137]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' #Data url
cpc = pd.read_html(url) #Pandas read 
df = cpc[0] #Get table
df.head() #Show table first values

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<p>2. Cleaning up the dataframe.</p>
<p>Drop rows where the 'Borough' values are 'Not assigned'.</p>

In [139]:
#Get index list of rows where borough is not assigned
index = df[df['Borough'] == 'Not assigned'].index

#Drop all undesired rows
df.drop(index, inplace = True)
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


3. Content Check
Check dataframe values for 'Not assigned' and repeated Postal Codes.

In [167]:
c1 = (df['Postal Code'].value_counts() > 1).value_counts() #Check if there are repeated postal codes
c2 = (df['Borough'] == 'Not Assigned').value_counts() #Check for not assigned values on Borough
c3 = (df['Neighborhood'] == 'Not Assigned').value_counts() #Check for not assigned values on Neighborhood

print('There are no postal codes repeated:')
print(c1)
print('There are no Borough with Not Assigned value:')
print(c2)
print('There are no Neighborhood with Not Assigned value:')
print(c3)

There are no postal codes repeated:
False    103
Name: Postal Code, dtype: int64
There are no Borough with Not Assigned value:
False    103
Name: Borough, dtype: int64
There are no Neighborhood with Not Assigned value:
False    103
Name: Neighborhood, dtype: int64


<p><h2>Second Part</h2></p>
<p><h3>Neighborhood Coordinates</h3></p>
<p>Read the provided csv file with the postal codes and their respective coordinates and merge the neighborhood and coordinate dataframes.</p>
<p>1. Read csv file.</p>

In [176]:
#Get dataframe with pandas
df_nc = pd.read_csv('http://cocl.us/Geospatial_data')
df_nc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


2. Merge both dataframes with 'Postal Code' as main colunm.

In [198]:
df_info = pd.merge(df, df_nc, on = 'Postal Code')
df_info.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


<p><h2>Third Part</h2></p>
<p><h3>Clustering Neighborhoods</h3></p>
<p>Filter and plot Toronto map. Cluster and analyse venues provided by Foursquare</p>
<p><h3>Toronto Map</h3></p>
<p><h4>Geolocator</h4></p>

In [264]:
#Create geolocator for Toronto
address = 'Toronto, CA'
geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

<h4>Plot Full Toronto Map</h4>

In [271]:
#First Toronto map with all neighborhoods

##Create map with folium library and the latitude and longitude variables
map_canada = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_info['Latitude'], 
                                           df_info['Longitude'],
                                           df_info['Borough'],
                                           df_info['Neighborhood']):
    
    #Maker display info
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    
    #Markers appearance
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = '#FF0000',
        fill = True,
        fill_color = 'white',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_canada)  
    
map_canada

<p><h4>Filter Data</h4></p>
<p>As proposed by the assignment, to simplify the analysis, lets filter the neighborhood dataframe by Boroughs containg the word 'Toronto'</p>

In [272]:
#filter data
toronto_data = df_info[df_info['Borough'].str.contains('Toronto')].reset_index(drop=True)

<h4>Plot Filtered Map</h4>

In [301]:
#Create filtered map of Toronto using latitude and longitude values

#There is a '+ 0.02' to the latitude, it is to better center the map, only for visualization
map_toronto = folium.Map(location=[latitude + 0.02, longitude], zoom_start=12)

#Add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], 
                                           toronto_data['Longitude'],
                                           toronto_data['Borough'],
                                           toronto_data['Neighborhood']):
    
    #Marker display info
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html = True)
    
    #Markers appearance
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = '#FF0000',
        fill = True,
        fill_color = 'white',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_toronto)  
    
map_toronto

<p><h3>Foursquare Data Aquisition and Processing</h3></p>
<p><h4>Foursquare ID&Secret</h4></p>
<p>I'll delete these before submitign this assignment.</p>

In [293]:
CLIENT_ID = '4TA3CJZNAX3YAOMWZ2ASBNZGULEUZI5E5OABCFPWXRV1UATU' #Foursquare ID
CLIENT_SECRET = 'TJGOQELC1CGZQSVX0QIQNHW3XCOXQJPN1OBSJDIYGLN1KRZX' #Foursquare Secret
VERSION = '20180605' #Foursquare API version

<p><h4>Define Search Functions</h4></p>
<p>Let's copy some of the functions from the New York notebook</p>

In [247]:
#Define a function to read a series of names and coordinates and
#lists the first 100 venues located inside a radius of 500m from the neighborhood center. 

def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<p><h4>Data Acquisiton</h4></p>

In [297]:
#Execute previous function and get a dataframe with all the venues from Toronto neighborhoods
toronto_venues = getNearbyVenues(names = toronto_data['Neighborhood'],
                                   latitudes = toronto_data['Latitude'],
                                   longitudes = toronto_data['Longitude']
                                  )

In [300]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub


<p><h3>Data Processing</h3></p>
<p>Get all venues categories</p>

In [325]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
#fixed_columns = [toronto_onehot.columns[toronto_onehot.index('Neighborhood')]] + list(toronto_onehot.columns[:toronto_onehot.index('Neighborhood')])
#toronto_onehot = toronto_onehot[fixed_columns]

#reorder columns with the neighborhood as the first one
col_list = list(toronto_onehot) #Get list of columns headers
col_list.insert(0, col_list.pop(col_list.index('Neighborhood'))) #Remove 'Neighborhood' column and reinsert it at 0 
toronto_onehot = toronto_onehot.loc[:, col_list] #Reorder dataframe

toronto_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

<p><h4>Filter</h4></p>
<p>I will filter the venues by the 'Restaurant' tag so I can analyze the neighborhoods which have a greater concentration of a particular couisine.</p>

In [342]:
#Get list of columns headers taht has 'Restaurant' in it
cols_rest = toronto_onehot.columns.str.contains('Restaurant|Neighborhood')
toronto_rest = toronto_onehot.loc[:,cols_rest]
toronto_rest.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Colombian Restaurant,Comfort Food Restaurant,Cuban Restaurant,Doner Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
